# Implementation of the Lee and Sun Spectral Sparsifier

In [1]:
import networkx as nx #This library is used to manage graph objects
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as LA
import scipy as sp
import random as rd

## The following functions return different kind of matrices associated to graphs

In [3]:
#receive an integer n
#return an undirected unweighted complete graph of n vertices
def get_complete_random_graph(n):
    Kn = nx.complete_graph(n)
    return Kn

#receive a graph G
#return the adjacency matrix of G as numpy array object
def get_adjacency_matrix(G):
    A = nx.adjacency_matrix(G)
    return A.toarray()

#receive a graph G
#return the Laplacian matrix of G as numpy array object
def get_laplacian_matrix(G):
    L = nx.laplacian_matrix(G)
    return L.toarray()

#receive a graph G
#return the Signed Incidence matrix of G as numpy array object
def get_signed_incidence_matrix(G):
    B = nx.incidence_matrix(G, oriented=True)
    return B.toarray()

def get_weight_diagonal_matrix(G):
    pass

#receive a graph G
#returns the vectors corresponding to every edge of G (the column vectos of the signed incidence matrix)
def get_rank_one_laplacian_decomposition(G):
    B = nx.incidence_matrix(G, oriented=True)
    m = G.number_of_edges()
    Vs = [] #vectors vi
    for i in range(m):
        u = B[:,i]
        Vs.append(u.toarray())
    return Vs

## The following functions are from the article "Constructing linear-sized spectral sparsification in almost-linear time"

In [4]:
#This function computes the relative effective resistance of the vectors Vi's
#receive the following parameters: 
#                        a set of vectos $$\{v_i\}_{i\leq m}$$ such that $$sum_{i=1}^m v_iv_i^T = I$$
#                        a matrix $A\in \mathrb{A}^{n\times n}$
#                        the actual barriers $u$ and $l$
def compute_Rs(Vs, A, u, l):
    Rs = []
    m = len(Vs)#number of Vi vectors (edges of the graph)
    n = A.shape[0]#number vertices of the graph
    I = np.identity(n)
    for i in range(m):
        Vi = Vs[i]
        maux1 = LA.inv((u*I-A))
        maux2 = LA.inv((A-l*I))
        Ri = np.dot(Vi.transpose(), np.dot(maux1, Vi)) + np.dot(Vi.transpose(), np.dot(maux2, Vi))
        Rs.append(float(Ri))
    return Rs

#This function computes the constant Nj that it
#receive the following parameters
#                        the sum of all effective resistance of all vectors $v_i$
#                        a $n\times n$ matrix $A$
#                        the actual barriers $u$ and $l$
#                        an integer $q\geq 10$
def get_constant_Nj(sumRs, A, u, l, q):
    m = len(Vs)
    n = A.shape[0]
    I = np.identity(n)
    eigmin = min(min(LA.eigvals(u*I-A)), min(LA.eigvals(A-l*I)))
    return int((sumRs*eigmin/(n**(2/q))))


#returns \Delta_u_j
def get_Delta_uj(eps, q, Nj, Rs):
    u = (1+(3*eps))*(eps*Nj/(q*sum(Rs)))
    print(u)
    return u

#returns \Delta_l_j
def get_Delta_lj(eps, q, Nj, Rs):
    l = (1-(3*eps))*(eps*Nj/(q*sum(Rs)))
    print(l)
    return l

#return the set of vectors V from the theorem and algorithm
def get_Vs(Lpnh, Us):
    Vs = []
    for u in Us:
        Vs.append(np.matmul(Lpnh, u))
    return Vs

#return a matrix to the power of x. It uses the diagonalization method
def get_matrix_raise_power_of_x(L, x):
    eigvals, eigvecs = LA.eigh(L)
    eigvals_power_x = [eigval**(x) if eigval >= 5e-10 else 0 for eigval in eigvals]
    Lambda = np.diag(eigvals_power_x)
    return np.matmul(np.matmul(eigvecs, Lambda), eigvecs.transpose())

#return sampled vectors according to a probability distribution over every vector (every vector represent an edge of the graph)
def get_Nj_sample_vectors(Rs, sumRs, Nj):
    m = len(Rs)
    p = [Rs[i]/sumRs for i in range(m)]
    return np.random.choice(m, Nj,p)
    

## From here we set our parameters and code the spectral sparsifier algorithm

In [5]:
#This lines of code define the graph G and compute the necessary matrices for getting a spectral sparsifier
n = 15
G = get_complete_random_graph(n)
L = get_laplacian_matrix(G)
Us = get_rank_one_laplacian_decomposition(G)
Lpnh = sp.linalg.sqrtm(sp.linalg.pinvh(L))
Vs = get_Vs(Lpnh, Us)
m = len(Vs)

In [7]:
# This is the implementation of the spectral sparsifier
j = 0
eps = 1/120
q = 10
l = -(2*n)**(1/q)
u = (2*n)**(1/q)
MAX_ITER = 10
Nj = 1
A = np.zeros((n,n))
Ss = np.zeros(m)
while(u-l <= 4*(2*n)**(1/q) and j <= MAX_ITER and Nj != 0):
    print("While condition: u-l= "+str(u-l)+"\t"+str(4*(2*n)**(1/q)))
    W = np.zeros((n,n))
    Rs = compute_Rs(Vs, A, u, l)
    sumRs = sum(Rs)
    print("Sum Rs: "+str(sumRs))
    Nj = get_constant_Nj(sumRs, A, u, l, q)
    print("Nj: "+str(Nj))
    indices = get_Nj_sample_vectors(Rs, sumRs, Nj)
    for i in indices:
        t = (eps/q)*(1/Rs[i])
        Ss[i] += t 
        W = W + t*np.outer(Vs[i], Vs[i])
    A = A + W
    u = u + get_Delta_uj(eps, q, Nj, Rs)
    print("uj: "+str(u))
    l = l + get_Delta_lj(eps, q, Nj, Rs)
    print("lj: "+str(l))
    print("Iteracion: "+str(j)+"\n")
    j = j+1
print(A)

While condition: u-l= 2.810231652967292	5.620463305934584
Sum Rs: 19.927182850164783
Nj: 16
0.0006858303438789218
uj: 1.405801656827525
0.0006523752051531207
lj: -1.404463451278493
Iteracion: 0

While condition: u-l= 2.8102651081060177	5.620463305934584
Sum Rs: 19.926949507687752
Nj: 16
0.0006858383748799138
uj: 1.406487495202405
0.0006523828443979669
lj: -1.403811068434095
Iteracion: 1

While condition: u-l= 2.8102985636364997	5.620463305934584
Sum Rs: 19.926714791197245
Nj: 16
0.0006858464533603906
uj: 1.4071733416557652
0.0006523905288062252
lj: -1.4031586779052887
Iteracion: 2

While condition: u-l= 2.810332019561054	5.620463305934584
Sum Rs: 19.926483519130905
Nj: 16
0.0006858544134767005
uj: 1.407859196069242
0.0006523981006241786
lj: -1.4025062798046646
Iteracion: 3

While condition: u-l= 2.8103654758739065	5.620463305934584
Sum Rs: 19.926250996552536
Nj: 16
0.0006858624168204621
uj: 1.4085450584860624
0.0006524057135609274
lj: -1.4018538740911037
Iteracion: 4

While condition: 

In [6]:
#Here we compute the spectral sparsifier of G by multiplying its weighted edge matrix times the reweighting factors
def compute_spectral_sparsifier(G, Ss):
    B = get_signed_incidence_matrix(G)
    W = np.diag(Ss)
    return np.matmul(B, np.matmul(W, B.transpose()))